In [ ]:

from data_generators import Payload, MIP_Solver
import uuid    
import pickle
import os
import pathlib
import shutil

# pickle data classes with train & valid data

# The data will be simulated for the following cases:
# - low bundle noise
# - low bundle availability
# - low store inventory
# - high need

def create_or_replace_folder(folder):
    
    if not os.path.exists(folder):
        print(f'{folder} does not exist, creating folder')
        os.mkdir(dfolder)
    
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

            
create_or_replace_folder('data/payloads/train/')
create_or_replace_folder('data/payloads/valid/')
create_or_replace_folder('data/mip_outputs/train/')
create_or_replace_folder('data/mip_outputs/valid/')

for seed in range(0,200):
    
    uniq_id = uuid.uuid4()
    
    payload = Payload(seed=seed, 
                    uniq_id=uniq_id,
                    dataset='train',
                    bundle_noise=0,
                    bundle_avail_lb=1,
                    bundle_avail_ub=20,
                    current_inv_lb=0,
                    current_inv_ub=10,
                    target_inv_noise_lb=-5,
                    target_inv_noise_ub=40)
    
    
    with open(f'data/payloads/train/{uniq_id}.pkl', 'wb') as f:
        pickle.dump(payload,f)
  
        
# valid

for seed in range(200,300):
    
    uniq_id = uuid.uuid4()
    
    payload = Payload(seed=seed, 
                    uniq_id=uniq_id,    
                    dataset='valid',
                    bundle_noise=0,
                    bundle_avail_lb=1,
                    bundle_avail_ub=20,
                    current_inv_lb=0,
                    current_inv_ub=10,
                    target_inv_noise_lb=-5,
                    target_inv_noise_ub=40)
    
    
    with open(f'data/payloads/valid/{uniq_id}.pkl', 'wb') as f:
        pickle.dump(payload,f)
        
        
def payload_data_generator(dir_path):

    out_list = os.listdir(dir_path)
    for file_nm in out_list:
        file_path = pathlib.PurePath(dir_path, file_nm)
        print(file_path)
        with open(file_path, 'rb') as f:
            data = pickle.load(f)
        yield data

        
def save_mip_outputs(data_gen):
    
    for payload in data_gen:
    
        mip_solver = MIP_Solver(payload=payload)

        with open(f'data/mip_outputs/{payload.dataset}/{payload.uniq_id}.pkl', 'wb') as f:
            pickle.dump(mip_solver,f)

            

train_data_gen = payload_data_generator('data/payloads/train')
valid_data_gen = payload_data_generator('data/payloads/valid')

save_mip_outputs(train_data_gen)
save_mip_outputs(valid_data_gen)
    